In [123]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import plotly as py 
import plotly.figure_factory as ff 
import plotly.express as px
import requests
import copy
from plotly.offline import plot
import plotly.graph_objects as go

In [124]:
pip install plotly==4.8.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
py.__version__

'4.8.1'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
raw_data = pd.read_csv('raw_data.csv')
raw_data1 = pd.read_csv('raw_data1.csv')
raw_data2 = pd.read_csv('raw_data2.csv')
raw_data3 = pd.read_csv('raw_data3.csv')
raw_data4 = pd.read_csv('raw_data4.csv')
raw_data5 = pd.read_csv('raw_data5.csv')
case_time_series = pd.read_csv('case_time_series.csv')
death_and_recovered = pd.read_csv('death_and_recovered.csv')
#death_and_recovered1 = pd.read_csv('/content/drive/My Drive/homework/data/death_and_recovered1.csv')
#death_and_recovered2 = pd.read_csv('/content/drive/My Drive/homework/data/death_and_recovered2.csv')
district_wise = pd.read_csv('district_wise.csv')
sources_list = pd.read_csv('sources_list.csv')
state_wise = pd.read_csv('state_wise.csv')
state_wise_daily = pd.read_csv('state_wise_daily.csv')
statewise_tested_numbers_data = pd.read_csv('statewise_tested_numbers_data.csv')
tested_numbers_icmr_data = pd.read_csv('tested_numbers_icmr_data.csv')
raw_data6 = pd.read_csv('travel_history.csv')
'''
print(type(raw_data6))
raw_data6 = json.loads(raw_data6)
raw_data6 = pd.read_dict('/content/drive/My Drive/homework/data/raw_data6.json')
raw_data6 = pd.DataFrame.from_json(raw_data6, orient='index')
raw_data6.reset_index(level=0, inplace=True)
'''

"\nprint(type(raw_data6))\nraw_data6 = json.loads(raw_data6)\nraw_data6 = pd.read_dict('/content/drive/My Drive/homework/data/raw_data6.json')\nraw_data6 = pd.DataFrame.from_json(raw_data6, orient='index')\nraw_data6.reset_index(level=0, inplace=True)\n"

In [7]:
merge_raw_data_1 = raw_data1.append(raw_data2)

In [8]:
merge_raw_data_2 = raw_data3.append(raw_data4)
merge_raw_data_2= merge_raw_data_2.append(raw_data5)

In [9]:
col=merge_raw_data_2.columns.tolist()
merge_raw_data_2['Num cases'] = merge_raw_data_2['Num Cases'] 
#merge_raw_data_2.info()

In [50]:
mask = ['Patient Number','State Patient Number', 'Date Announced', 'Age Bracket', 'Gender', 'Detected City', 'Detected District', 'Detected State', 'State code', 'Current Status', 'Contracted from which Patient (Suspected)', 'Notes', 'Nationality', 'Type of transmission', 'Status Change Date', 'Num cases']
merge_raw_data_1 = merge_raw_data_1[mask]
merge_raw_data_2 = merge_raw_data_2[mask]
raw_data = merge_raw_data_1.append(merge_raw_data_2)
#raw_data.info()
print(raw_data.head())
print(raw_data.info())

   Patient Number State Patient Number Date Announced Age Bracket Gender  \
0             1.0             KL-TS-P1     30/01/2020          20      F   
1             2.0             KL-AL-P1     02/02/2020         NaN    NaN   
2             3.0             KL-KS-P1     03/02/2020         NaN    NaN   
3             4.0                DL-P1     02/03/2020          45      M   
4             5.0                TS-P1     02/03/2020          24      M   

              Detected City Detected District Detected State State code  \
0                  Thrissur          Thrissur         Kerala         KL   
1                 Alappuzha         Alappuzha         Kerala         KL   
2                 Kasaragod         Kasaragod         Kerala         KL   
3  East Delhi (Mayur Vihar)        East Delhi          Delhi         DL   
4                 Hyderabad         Hyderabad      Telangana         TG   

  Current Status Contracted from which Patient (Suspected)  \
0      Recovered              

# **印度各州隨時間的總感染數變化 - state_wise_daily資料**

In [11]:
df_tc = copy.deepcopy(state_wise_daily)

### 取狀態為 confirmed cases 的資料

In [12]:
df_tc_conf = df_tc.loc[df_tc['Status']=='Confirmed']

In [13]:
df_tc_conf.reset_index(inplace=True)
df_tc_conf.drop(columns=['index'],axis=1,inplace=True)

### 計算各州隨時間累積總確診數

In [14]:
for col in df_tc_conf.columns[2:]:
    for i in range(81):
        df_tc_conf[col][i+1] = df_tc_conf[col][i]+df_tc_conf[col][i+1]

In [15]:
df_tc_conf.drop(columns=['TT','UN'],axis=1,inplace=True)

### 利用 melt 轉換各州欄位為單一欄位"State" 

In [16]:
df_tc_conf_melt = df_tc_conf.melt(id_vars=['Date','Status'], var_name = "State", value_name = "Total Cases")

### 替換州碼，讓後續地圖可以透過州名抓位置

In [17]:
state_code_switch = {
    'AP':'Andhra Pradesh','AR':'Arunachal Pradesh','AS':'Assam','BR':'Bihar','CT':'Chhattisgarh','GA':'Goa','GJ':'Gujarat','HR':'Haryana','HP':'Himachal Pradesh',
    'JK':'Jammu and Kashmir','JH':'Jharkhand','KA':'Karnataka','KL':'Kerala','MP':'Madhya Pradesh','MH':'Maharashtra','MN':'Manipur','ML':'Meghalaya','MZ':'Mizoram',
    'NL':'Nagaland','OR':'Orissa','PB':'Punjab','RJ':'Rajasthan','SK':'Sikkim','TN':'Tamil Nadu','TR':'Tripura','UT':'Uttarakhand','UP':'Uttar Pradesh','WB':'West Bengal',
    'TN':'Tamil Nadu','TR':'Tripura','AN':'Andaman and Nicobar Islands','CH':'Chandigarh','DH':'Dadra and Nagar Haveli','DD':'Daman and Diu','DL':'Delhi','LD':'Lakshadweep',
    'DN':'Dadra and Nagar Haveli and Daman and Diu','TG':'Telangana','PY':'Puducherry','LA':'Ladakh'
}

In [18]:
df_tc_conf_melt['State'] = df_tc_conf_melt['State'].map(state_code_switch)

## 新增月份日期欄位

In [19]:
df_tc_conf_melt['Month']=df_tc_conf_melt['Date'].str.split('-',expand=True)[1]
df_tc_conf_melt['Day']=df_tc_conf_melt['Date'].str.split('-',expand=True)[0]

In [20]:
df_conf_m = df_tc_conf_melt.loc[(df_tc_conf_melt['Day']=='30')|
                                ((df_tc_conf_melt['Day']=='15')&(df_tc_conf_melt['Month']=='Apr'))|
                                ((df_tc_conf_melt['Day']=='15')&(df_tc_conf_melt['Month']=='May'))|
                                ((df_tc_conf_melt['Day']=='03')&(df_tc_conf_melt['Month']=='Jun'))]

In [21]:
df_conf_m.head(3)

,Date,Status,State,Total Cases,Month,Day
16,30-Mar-20,Confirmed,Andaman and Nicobar Islands,10,Mar,30
32,15-Apr-20,Confirmed,Andaman and Nicobar Islands,11,Apr,15
47,30-Apr-20,Confirmed,Andaman and Nicobar Islands,33,Apr,30


-------

# **視覺化 - 印度各州隨時間總感染數變化**

### 抓取印度地圖 geojson

In [22]:
indiajson_url = 'https://raw.githubusercontent.com/Subhash9325/GeoJson-Data-of-Indian-States/master/Indian_States'
india_states_geo = requests.get(indiajson_url).json()

### 畫圖

In [22]:
fig =  px.choropleth_mapbox(
                    data_frame=df_conf_m, 
                    geojson=india_states_geo, 
                    locations='State',
                    #featureidkey='properties.NAME_1',
                    color='Total Cases',
                    range_color=[0,75000],
                    color_continuous_scale='Reds',
                    hover_name="State",
                    mapbox_style="carto-positron",
                    center={"lat": 23.48, "lon": 80.12},
                    zoom = 3,
                    animation_frame='Date'
                   )
fig.show()


ValueError: ignored

## 調查各個省分的狀態


In [110]:
complete = pd.read_csv('Covid-19-Cornora-Virus-in-India-Dataset/datasets_549966_1242662_complete.csv')
patients = pd.read_csv('Covid-19-Cornora-Virus-in-India-Dataset/patients_data.csv')
state_latest = pd.read_csv('Covid-19-Cornora-Virus-in-India-Dataset/datasets_549966_1242662_state_level_latest.csv')
daily = pd.read_csv('Covid-19-Cornora-Virus-in-India-Dataset/datasets_549966_1242662_nation_level_daily.csv')
test_day = pd.read_csv('Covid-19-Cornora-Virus-in-India-Dataset/datasets_549966_1242662_tests_day_wise.csv')
print(complete.info())
print(patients.info())
print(state_latest.info())
print(daily.info())
print(test_day.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       2951 non-null   object 
 1   Name of State / UT         2951 non-null   object 
 2   Latitude                   2951 non-null   float64
 3   Longitude                  2951 non-null   float64
 4   Total Confirmed cases      2951 non-null   int64  
 5   Death                      2951 non-null   int64  
 6   Cured/Discharged/Migrated  2951 non-null   int64  
 7   New cases                  2951 non-null   int64  
 8   New deaths                 2951 non-null   int64  
 9   New recovered              2951 non-null   int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 230.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89694 entries, 0 to 89693
Data columns (total 23 columns):
 #   Column                                

c:\users\rexhu\anaconda3\envs\test\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning:

Columns (4,13,14,15,19) have mixed types.Specify dtype option on import or set low_memory=False.



In [107]:
new_cases = complete['New cases']
new_deaths = complete['New deaths']
new_recovered = complete['New recovered']
death = complete['Death']
Cured = complete['Cured/Discharged/Migrated']
total_cases = complete['Total Confirmed cases']
daily['Total Active'] = daily['Total Confirmed'] - daily['Total Recovered'] - daily['Total Deceased']
daily['Deceased Rate'] = round(daily['Total Deceased']/daily['Total Confirmed']*100, 2)
daily['Recovered Rate'] = round(daily['Total Recovered']/daily['Total Confirmed']*100, 2)
daily['Deceased/Recovered Rate'] = round(daily['Total Deceased']/daily['Total Recovered']*100, 2)
state_latest['Mortality Rate'] = round(state_latest['Deaths']/state_latest['Confirmed']*100, 2)
state_latest['Recovery Rate'] = round(state_latest['Recovered']/state_latest['Confirmed']*100, 2)
print(daily.describe())

       Daily Confirmed  Total Confirmed  Daily Recovered  Total Recovered  \
count       135.000000       135.000000       135.000000       135.000000   
mean       2293.325926     48711.925926      1142.511111     20551.000000   
std        3247.379759     79097.439983      1942.994519     37943.537045   
min           0.000000         1.000000         0.000000         0.000000   
25%           1.500000        17.000000         0.000000         3.000000   
50%         573.000000      4777.000000        65.000000       394.000000   
75%        3597.000000     65020.500000      1527.000000     20135.500000   
max       11306.000000    309599.000000     11735.000000    154239.000000   

       Daily Deceased  Total Deceased   Total Active  Deceased Rate  \
count      135.000000      135.000000     135.000000     135.000000   
mean        65.844444     1439.785185   26721.140741       1.955630   
std         92.833896     2254.839099   39321.857402       1.404522   
min          0.000000 

In [87]:
latest = state_latest.sort_values('Confirmed', ascending=False).reset_index(drop=True)
# remove 'State Unassigned' row
latest = latest[latest['State']!='State Unassigned']
# rearrange columns
latest = latest.loc[:, ['State', 'Confirmed', 'Active', 'Deaths', 'Mortality Rate', 
                        'Recovered', 'Recovery Rate']]
# background color
latest.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active'])\
    .background_gradient(cmap="Greens", subset=['Recovered', 'Mortality Rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Recovery Rate'])


,State,Confirmed,Active,Deaths,Mortality Rate,Recovered,Recovery Rate
0,Total,310156,146342,8895,2.870000,154896,49.940000
1,Maharashtra,101141,49628,3717,3.680000,47796,47.260000
2,Tamil Nadu,40698,18284,367,0.900000,22047,54.170000
3,Delhi,36824,22212,1214,3.300000,13398,36.380000
4,Gujarat,22562,5645,1416,6.280000,15501,68.700000
5,Uttar Pradesh,12616,4642,365,2.890000,7609,60.310000
6,Rajasthan,12186,2736,275,2.260000,9175,75.290000
7,Madhya Pradesh,10443,2802,440,4.210000,7201,68.960000
8,West Bengal,10244,5587,451,4.400000,4206,41.060000
10,Karnataka,6516,2995,79,1.210000,3440,52.790000


In [169]:
def plot_daily(col, hue):
    fig = px.bar(daily, x="Date", y=col, title=col,color_discrete_sequence=[hue])
    #fig.add_trace(go.Scatter(x=['25 March'], y=[i for i in range(300000)],mode = 'lines', line=dict(dash='dash', color=('#F0F00F')))
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [170]:
plot_daily('Total Confirmed', '#000000')
print(daily.head())

           Date  Daily Confirmed  Total Confirmed  Daily Recovered  \
0   30 January                 1                1                0   
1   31 January                 0                1                0   
2  01 February                 0                1                0   
3  02 February                 1                2                0   
4  03 February                 1                3                0   

   Total Recovered  Daily Deceased  Total Deceased  
0                0               0               0  
1                0               0               0  
2                0               0               0  
3                0               0               0  
4                0               0               0  


In [94]:
plot_daily('Total Deceased', '#000000')

In [96]:
plot_daily('Total Recovered', '#000000')

In [71]:
plot_daily('Cured/Discharged/Migrated', '#000000')

In [102]:
temp = daily[['Date','Total Active', 'Total Recovered', 'Total Deceased']]
temp = temp.melt(value_vars=['Total Recovered', 'Total Deceased', 'Total Active'],
                 id_vars="Date", var_name='Case', value_name='Count')
fig=px.bar(temp, x='Date', y='Count', color='Case', title='Case over the time',color_discrete_sequence = ['#21bf73', '#ff2e63', '#393e46'] )
fig.show()

In [103]:
px.line(daily, x='Date', y='Deceased Rate', title='Deceased Rate')

In [106]:
px.line(daily, x='Date', y='Recovered Rate', title='Recovered Rate')

In [114]:
px.line(test_day, x='Update Time Stamp', y='Tests per million', title='Positive Rate')

In [117]:
px.scatter(state_latest[state_latest['Confirmed']>10000], x='Confirmed', y='Deaths', 
           color='State', size='Confirmed', text='State', 
           log_x =True, log_y=True, title='Confirmed vs Death')

In [120]:
px.histogram(patients, x='Age Bracket',title='Distribution of ages of confirmed patients', nbins=50)